# Customer Churn Prediction 💁‍♀️

### Data Queries

In [1]:
# Load required libraries
import pandas as pd
from sqlalchemy import create_engine
import config

## Database Configuration

### MySQL Connection

In [2]:
# Create a connection to MySQL database
engine = create_engine('mysql+mysqlconnector://' + config.username + ':' + config.password + '@localhost:3306/' + config.database)

### Data Loading

In [ ]:
# Load data into MySQL database
csv_files = {
    "customers": "../Data/customers.csv",
    "accountcharges": "../Data/AccountCharges.csv",
    "accountfeatures": "../Data/AccountFeatures.csv",
    "internetservices": "../Data/InternetServices.csv",
    "phoneservices": "../Data/PhoneServices.csv",
}

for table, file in csv_files.items():
    df = pd.read_csv(file)
    df.to_sql(table, con=engine, if_exists="append", index=False)

7043

## Data Querying

### Customers
The EDA suggested churn is most likely for customers who are senior citizens, do not have a partner and do not have dependants. Below the study of this demosgraphic's churn rate.

In [3]:
query = '''
SELECT 
    (COUNT(customers.CustomerID) * 100 / 
        (SELECT COUNT(CustomerID) 
         FROM customers 
         WHERE SeniorCitizen = 1 
           AND Partner = 'No' 
           AND Dependents = 'No')
    ) AS 'Churn Percentage',
COUNT(customers.CustomerID) AS 'Number of Customers'
FROM customers
JOIN accountfeatures ON customers.CustomerID = accountfeatures.CustomerID
WHERE customers.SeniorCitizen = 1
    AND customers.Partner = 'No'
    AND customers.Dependents = 'No'
    AND accountfeatures.Churn = 1
'''
df = pd.read_sql(query, con=engine)
print(df)

   Churn Percentage  Number of Customers
0           49.1979                  276


### Account Features

Below a review of the avergae customer tenure by churn status

In [20]:
query = '''
SELECT
accountfeatures.Churn,
AVG(Tenure) as 'Average Tenure'
FROM accountfeatures
GROUP BY Churn
'''
df = pd.read_sql(query, con=engine)
print(df)

   Churn  Average Tenure
0      0         37.5700
1      1         17.9791


EDA also suggests churn is most likely for customers whose contract is month-to-month, payment is by electronic check and tenure is less than 30 months. Below the study of this group's churn rate.

In [5]:
query = '''
SELECT 
(COUNT(customerID) * 100 /
    (SELECT COUNT(customerID)
        FROM accountfeatures
        WHERE Contract = 'Month-to-month'
            AND PaymentMethod = 'Electronic check'
            AND Tenure <= 30)) AS 'Churn Percentage',
COUNT(customerID) AS 'Number of Customers'
FROM accountfeatures
WHERE Contract = 'Month-to-month' and
    PaymentMethod = 'Electronic check' and
    Tenure <= 30 and
    Churn = 1
'''
df = pd.read_sql(query, con=engine)
print(df)

   Churn Percentage  Number of Customers
0           58.6255                  836


### Internet Services

Similarly, the EDA suggests that fiber optic internet services are the most likely to churn. Below the customer features analysis for this group.

In [7]:
query = '''
SELECT
(COUNT(internetservices.CustomerID) * 100 /
    (SELECT COUNT(internetservices.CustomerID)
        FROM internetservices
        JOIN accountfeatures ON internetservices.CustomerID = accountfeatures.CustomerID
        WHERE internetservices.InternetService = 'Fiber optic'
            AND accountfeatures.Churn = 1)) AS 'Churn Percentage',
COUNT(internetservices.CustomerID) AS 'Number of Customers'
FROM internetservices
JOIN accountfeatures ON internetservices.CustomerID = accountfeatures.CustomerID
WHERE internetservices.InternetService = 'Fiber optic'
    AND accountfeatures.Churn = 1
    AND accountfeatures.Contract = 'Month-to-month'
    AND accountfeatures.PaymentMethod = 'Electronic check'
    AND accountfeatures.Tenure <= 30
'''
df = pd.read_sql(query, con=engine)
print(df)

   Churn Percentage  Number of Customers
0           49.9614                  648


### Account Charges

EDA revealed that customers who churn have higher monthly costs and lower total costs. On the total costs front, this view aligns with the previous analysis on account features, given churned customers have a considerably lower tenure than customer who do not churn. Hence, total costs are added across a shorter period of time. On the other hand, re monthly costs, below the corresponding analysis.

In [21]:
query = '''
SELECT
    accountfeatures.Churn,
    AVG(accountcharges.MonthlyCharges) as 'Avg. Monthly Charges'
FROM accountcharges
JOIN accountfeatures ON accountcharges.CustomerID = accountfeatures.CustomerID
GROUP BY accountfeatures.Churn
'''
df = pd.read_sql(query, con=engine)
print(df)

   Churn  Avg. Monthly Charges
0      0               61.2882
1      1               74.4580


In [29]:
query = '''
SELECT 
    accountfeatures.Churn,
    COUNT(CASE WHEN internetservices.InternetService != 'No' THEN internetservices.CustomerID END) AS 'Has Internet Service',
    COUNT(CASE WHEN internetservices.StreamingMovies = 'Yes' THEN internetservices.CustomerID END) AS 'Has Movie Streaming',
    COUNT(CASE WHEN internetservices.StreamingTV = 'Yes' THEN internetservices.CustomerID END) AS 'Has TV Streaming',
    COUNT(CASE WHEN phoneservices.PhoneService != 'No' THEN phoneservices.CustomerID END) AS 'Has Phone Service',
    COUNT(CASE WHEN phoneservices.MultipleLines = 'Yes' THEN phoneservices.CustomerID END) AS 'Has Multiple Lines'
FROM accountfeatures
LEFT JOIN internetservices ON accountfeatures.CustomerID = internetservices.CustomerID
LEFT JOIN phoneservices ON accountfeatures.CustomerID = phoneservices.CustomerID
GROUP BY accountfeatures.Churn
'''
df = pd.read_sql(query, con=engine)
print(df)

   Churn  Has Internet Service  Has Movie Streaming  Has TV Streaming  \
0      0                  3761                 1914              1893   
1      1                  1756                  818               814   

   Has Phone Service  Has Multiple Lines  
0               4662                2121  
1               1699                 850  


Below an study of the number of customer by contract type and churn status. The analysis showcases that the majority of churn cases are concentrated in the month-to-month contract (likely the most expensive contract). Although customers who do not churn are also a majority month-to-month, the split across this type of contract along with one year and two year is much more balanced.

In [39]:
query = '''
SELECT 
Contract,
Churn,
COUNT(customerID) AS 'Number of Customers'
FROM accountfeatures
GROUP BY Contract, Churn
ORDER BY Contract
'''
df = pd.read_sql(query, con=engine)
print(df)

         Contract  Churn  Number of Customers
0  Month-to-month      0                 2220
1  Month-to-month      1                 1655
2        One year      0                 1307
3        One year      1                  166
4        Two year      0                 1647
5        Two year      1                   48
